Exercise 3 \\
Regression

In [207]:
#Installing Pyomo
!pip install -q pyomo

#Importing required packages
from pyomo.environ import *
import pandas as pd
import numpy as np

In [208]:
df = pd.read_csv('Lab_06_data.csv')

In [209]:
#Create model
model_ex3 = ConcreteModel()

In [210]:
#we declare a Python variable N denoting the number of decision variables
N = 6
# set of column indices for original decision variables
column_indices = np.arange(N)
model_ex3.a = Var(column_indices, domain=NonNegativeReals)

In [211]:
#we declare a Python variable M denoting the number of constraints
M = 12 
# set of row indices for original constraints
row_indices = np.arange(M)

In [212]:
#Variable declaration

In [213]:
model_ex3.b0 = Var(domain = NonNegativeReals)

In [214]:
model_ex3.b1 = Var()

In [215]:
model_ex3.b2= Var()

In [216]:
#y is for deviation
model_ex3.y= Var(range(0,6),domain = Reals)

In [217]:
#Add Objective
model_ex3.obj = Objective(expr=sum(model_ex3.y[i]**2 for i in column_indices),sense=minimize)

In [218]:
#Creating Constraintlist
model_ex3.constraints = ConstraintList()

In [219]:
for i in column_indices:
  model_ex3.constraints.add(expr=((df.iloc[i,1])-(model_ex3.b0)-(df.iloc[i,2]*model_ex3.b1)-(df.iloc[i,3]*model_ex3.b2) == model_ex3.y[i]) )

In [220]:
model_ex3.pprint()

3 Set Declarations
    a_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {1, 2, 3, 4, 5, 6}
    y_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {0, 1, 2, 3, 4, 5}

5 Var Declarations
    a : Size=6, Index=a_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals
          3 :     0 :  None :  None : False :  True : NonNegativeReals
          4 :     0 :  None :  None : False :  True : NonNegativeReals
          5 :     0 :  None :  None : Fals

In [221]:
%%bash
rm -fv ipopt*
apt install -q -yy libgfortran5
sleep 1
wget -q https://www.ieor.iitb.ac.in/files/faculty/amahajan/ipopt
wget -q https://www.ieor.iitb.ac.in/files/faculty/amahajan/test.nl
chmod u+rx ipopt

removed 'ipopt'
Reading package lists...
Building dependency tree...
Reading state information...
libgfortran5 is already the newest version (8.4.0-1ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


The above code also fetches an optimization problem in an '.nl' format. 'nl' format is suitable for solvers, not for humans. Lets call ipopt directly (without using any Python) to solve it.

In [222]:
!./ipopt test.nl



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      230

Total number of variables............................:       30
                     variables with only lower bounds:       30
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equ

In [223]:
opt_ipopt = SolverFactory('ipopt')
result = opt_ipopt.solve(model_ex3)
print(result)


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 6
  Number of variables: 9
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.13.2\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.032613277435302734
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [224]:
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [225]:
# display solution
print('The value of Objective Function is = ', model_ex3.obj())

The value of Objective Function is =  4291134028.8074546


In [226]:
print('Deviation :')
for i in column_indices:
  print('y(',i+1,') :' ,model_ex3.y[i].value) #y denotes deviation

Deviation :
y( 1 ) : 48762.44439737344
y( 2 ) : 14058.72696462612
y( 3 ) : -28366.606651133225
y( 4 ) : -26533.83816987928
y( 5 ) : -13331.391654310495
y( 6 ) : 5410.665113323435


Maximum deviation is 48762.44439737344 in this problem \\
Whereas in lab 6 Maximum deviation was 54272.727

Printing the details of decision variables

In [227]:
print('Decision Variables')
print('b[0] :', model_ex3.b0.value,'\nb[1] :', model_ex3.b1.value,'\nb[2] :', model_ex3.b2.value)

Decision Variables
b[0] : 104904.41643719553 
b[1] : 38.7020758313917 
b[2] : -193.32238932429613


In this problem the values of the decision variables are: \\
b[0]=104904.4164371955 \\
b[1]=38.702075831391646 \\
b[2]=-193.3223893242956 \\

In Lab 6 the values of the decision variables are : \\
b[0]=104181.82 \\
b[1]=67.272727 \\
b[2]=-356.36364


Observation :
Maximum deviation has decreases as we have used square method.